In [ ]:

# Set up the notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

In [ ]:

from FRVRS import (osp, np, nu, to_datetime, listdir, read_csv, csv, DataFrame)

In [8]:

# Find out if any files dated the same are actually jungle
from datetime import date

mask_series = file_stats_df.session_file_date.isin([date(2023, 9, 7), date(2023, 9, 11)]) & (file_stats_df.scenario_environment != 'jungle')
df = file_stats_df[mask_series]
session_uuid = df.session_uuid.squeeze()
print(df.shape) # (1, 11)
file_name = df.file_name.squeeze()
print(osp.abspath(osp.join('../data/logs', file_name)))
mask_series = (frvrs_logs_df.session_uuid == session_uuid) & ~frvrs_logs_df.patient_id.isnull()
patients_list = sorted(frvrs_logs_df[mask_series].patient_id.unique())
session_file_date = df.session_file_date.squeeze()
print(
    f'Doug,\n\nThere is a file called {file_name} that was dated {session_file_date.date()}'
    f' that only has {nu.conjunctify_nouns(patients_list)} in it - no military.'
    ' Was that session also in the jungle environment?'
)

(1, 11)
/mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv
Doug,

There is a file called v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv that was dated 2023-09-11 that only has Bob_1 Root, Bob_10 Root, Bob_12 Root, Bob_7 Root, Gary_16 Root, Gary_2 Root, Helga_14 Root, Helga_17 Root, Helga_6 Root, Helga_9 Root, Lily_0 Root, Lily_11 Root, Lily_15 Root, Lily_5 Root, Mike_13 Root, Mike_3 Root, Mike_4 Root, and Mike_8 Root in it - no military. Was that session also in the jungle environment?


In [34]:

# Create a missing demographics Excel file, delete the jungle entries and the responder name column
# and then send it to OSU with BigBear.ai and CACI CC’d
missing_columns_list = ['responder_type', 'site_name', 'encounter_layout']
mask_series = False
for cn in missing_columns_list: mask_series |= file_stats_df[cn].isnull()
mask_series &= (file_stats_df.scenario_environment != 'jungle')
columns_list = ['is_scene_aborted']
for cn in file_stats_df.columns:
    if (file_stats_df[mask_series][cn].nunique() > 1) and (cn not in missing_columns_list): columns_list.append(cn)
df = file_stats_df[mask_series][columns_list+missing_columns_list].drop(columns=['responder_name'])
df.is_scene_aborted = ''
df.to_excel('../saves/xlsx/missing_demographics.xlsx', index=False)
stamps_list = [ts.timestamp() for ts in df.sort_values('session_file_date').session_file_date]

# It would also be good to include in the text of your email the row sum total and grouped by date range "out of" total by date:
missing_columns_str = []
for cn in missing_columns_list:
    mask_series = ~file_stats_df[cn].isnull() & (file_stats_df[cn] != 'Unknown')
    values_list = np.random.choice(file_stats_df[mask_series][cn].unique(), size=2, replace=False).tolist()
    missing_columns_str.append(f'{cn} (' + ', '.join(values_list) + ', etc.)')
missing_columns_str = nu.conjunctify_nouns(missing_columns_str)
print(
    f'Guys,\n\nWe need you to fill the missing (as of 2/15) data in the {missing_columns_str}'
    f' columns of the attached spreadsheet.\n\nThere are {df.shape[0]} rows of missing information:'
)

# 53 from 3/15-3/16/2022
# 30 from 11/30-12/6/2022
# 23 from 3/7-3/15/2023
# 14 from 4/19-4/25/2023
# 19 from 5/1-5/23/2023
# 9 from 6/1-6/23/2023
# 2 from 7/27/2023
# 9 from 8/1-8/16/2023
file_count = 0
full_splits_list = nu.get_splits_list(
    [ts.timestamp() for ts in file_stats_df.sort_values('session_file_date').session_file_date], value_difference=24*60*60
)
for grouping in nu.get_splits_list(stamps_list, value_difference=24*60*60):
    from_date_obj = to_datetime(grouping[0], unit='s')
    from_date = from_date_obj.strftime('%m/%d')
    to_date_obj = to_datetime(grouping[-1], unit='s')
    to_date = to_date_obj.strftime('%m/%d/%Y')
    full_group = []
    for subgroup in full_splits_list:
        if grouping[0] in subgroup:
            full_group = subgroup
            break
    y = 'y'
    if full_group:
        y = len(full_group)
        file_count += y
    date_range = f'{from_date} to {to_date}'
    if (from_date_obj == to_date_obj): date_range = from_date
    print(f'{len(grouping)} out of {y} from {date_range}')

Dave and Doug,

We need you to fill the missing (as of 2/15) data in the responder_type (Paramedic, EMT-I, etc.), site_name (Union County, Plain Twp, etc.), and encounter_layout (First 11, DCEMS 11, etc.) columns of the attached spreadsheet.

There are 159 rows of missing information:
53 out of 53 from 03/15 to 03/16/2022
23 out of 23 from 11/30 to 12/02/2022
7 out of 7 from 12/06
10 out of 26 from 03/07 to 03/08/2023
12 out of 47 from 03/14 to 03/15/2023
3 out of 51 from 04/19 to 04/20/2023
11 out of 11 from 04/25
2 out of 2 from 05/01
7 out of 35 from 05/09 to 05/12/2023
8 out of 8 from 05/16
2 out of 2 from 05/22 to 05/23/2023
2 out of 11 from 06/08
2 out of 2 from 06/12
5 out of 11 from 06/23
2 out of 25 from 07/27
1 out of 26 from 08/01
4 out of 26 from 08/09 to 08/10/2023
4 out of 5 from 08/16
1 out of 6 from 09/11


In [51]:

# The 159 rows with missing data for: responder_type; site_name; encounter_layout
# Are those 159 rows part of the 389 CSV files Doug provided to you?
# Or do you have 389 and you are looking for an additional 159?
# I noticed that your lists of "x out of y" for date clusters; the ys add up to 377
# Is it possible that 389 - z = 377?
# where z = the jungle scenarios that shouldn't be in there
# Does that fully define z?
# I believe this was a miscommunication in the 11:30 meeting and I will like to make sure I understand the status.
import os

sub_directory = '../data/temp'
file_names_list = listdir(sub_directory)
print(
    f'The {len(file_names_list)} files in the "CSV Files Renamed by Date.zip" file that Doug uploaded to Data and'
    ' Videos (on Sep 23, 2023) contain ', end=''
)
session_uuids_list = []
for file_name in file_names_list:
    
    # Construct the full path to the file
    file_path = osp.join(sub_directory, file_name)
    
    # Attempt to read CSV file using pandas
    try:
        import pandas as pd
        version_number = '1.0'
        file_df = read_csv(file_path, header=None, index_col=False)
    
    # If unsuccessful, try using a reader
    except:
        version_number = '1.3'
        rows_list = []
        with open(file_path, 'r') as f:
            reader = csv.reader(f, delimiter=',', quotechar='"')
            for values_list in reader:
                if (values_list[-1] == ''): values_list.pop(-1)
                rows_list.append({i: v for i, v in enumerate(values_list)})
        file_df = DataFrame(rows_list)
    
    for session_uuid in file_df[3].unique():
        session_uuids_list.append(session_uuid)
print(f'{len(set(session_uuids_list))} unique UUIDs.')

The 385 files in the "CSV Files Renamed by Date.zip" file that Doug uploaded to Data and Videos (on Sep 23, 2023) contain 385 unique UUIDs.


In [55]:

# Create a missing demographics Excel file for the is_in_registry data
missing_columns_list = ['responder_type', 'site_name', 'encounter_layout']
mask_series = False
for cn in missing_columns_list: mask_series |= file_stats_df[cn].isnull()
mask_series &= file_stats_df.session_uuid.isin(session_uuids_list)
columns_list = []
for cn in file_stats_df.columns:
    if (file_stats_df[mask_series][cn].nunique() > 1) and (cn not in missing_columns_list): columns_list.append(cn)
columns_list += ['is_scene_aborted']
df = file_stats_df[mask_series][columns_list+missing_columns_list]
df.is_scene_aborted = ''
df.to_excel('../saves/xlsx/missing_demographics.xlsx', index=False)
stamps_list = [ts.timestamp() for ts in df.sort_values('session_file_date').session_file_date]

# It would also be good to include in the text of your email the row sum total and grouped by date range "out of" total by date:
missing_columns_str = []
for cn in missing_columns_list:
    mask_series = ~file_stats_df[cn].isnull() & (file_stats_df[cn] != 'Unknown')
    values_list = np.random.choice(file_stats_df[mask_series][cn].unique(), size=2, replace=False).tolist()
    missing_columns_str.append(f'{cn} (' + ', '.join(values_list) + ', etc.)')
missing_columns_str = nu.conjunctify_nouns(missing_columns_str)
print(
    f'Dave,\n\nOf the {len(file_names_list)} files in the "CSV Files Renamed by Date.zip" file that Doug uploaded to Data and'
    ' Videos (on Sep 23, 2023)'
    f' attached are the ones that have the missing data in the {missing_columns_str} columns.'
)

Dave,

Of the 385 files in the "CSV Files Renamed by Date.zip" file that Doug uploaded to Data and Videos (on Sep 23, 2023) attached are the ones that have the missing data in the responder_type (EMT-Basic, EM-RES1, etc.), site_name (DCEMS-RND 1, Madison Twp, etc.), and encounter_layout (First 11, DCEMS 11, etc.) columns.


In [30]:

# The 159 rows with missing data for: responder_type; site_name; encounter_layout
# Are those 159 rows part of the 389 CSV files Doug provided to you?
missing_columns_list = ['responder_type', 'site_name', 'encounter_layout']
mask_series = False
for cn in missing_columns_list: mask_series |= file_stats_df[cn].isnull()
mask_series &= file_stats_df.session_uuid.isin(session_uuids_list)
columns_list = ['responder_type', 'site_name', 'encounter_layout']
df = file_stats_df[mask_series]
print(
    f'{df.shape[0]} of those {len(set(session_uuids_list))} unique UUIDs are part of the 159 rows with missing data for'
    ' responder_type, site_name, and encounter_layout.'
)

134 of those 385 unique UUIDs are part of the 159 rows with missing data for responder_type, site_name, and encounter_layout.
